In [85]:
import pandas_ta as ta
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch #pytorch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [101]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df_x)

In [102]:
X = np.array(df_scaled[:, :-1])
y = df_scaled[:, -1]

In [103]:
X = X[:-24]
y = y[:-24]

In [105]:
def create_sequences(X, y, seq_len):
    X_seq = []
    Y_seq = []
    for i in range(0, len(X) -1 - seq_len):
        x_seq_row = []
        for j in range(i, i + seq_len):
            x_seq_row.append(X[j])
        X_seq.append(x_seq_row)
        Y_seq.append(y[j + 1])
    return X_seq, Y_seq

In [106]:
class Data(Dataset):
    def __init__(self, X, y):
        self.x=torch.tensor(X).float().to(DEVICE)
        self.y=torch.tensor(y).float().to(DEVICE)
        self.len=self.x.shape[0]
    def __getitem__(self,index):      
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len

In [107]:
X_seq, Y_seq = create_sequences(X, y, 24)


In [108]:
X_train, X_test, y_train, y_test = train_test_split(
    X_seq, Y_seq, test_size=0.2, shuffle=False)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.15, shuffle=False)

In [116]:
train_data = Data(X_train, y_train)
val_data = Data(X_val, y_val)
test_data = Data(X_test, y_test)

train_loader=DataLoader(dataset=train_data,batch_size=256)
val_loader=DataLoader(dataset=val_data,batch_size=256)
test_loader=DataLoader(dataset=test_data,batch_size=256)


In [117]:
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc =  nn.Linear(hidden_size, num_classes) #fully connected 1
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(DEVICE) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(DEVICE) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        out = hn[-1]
        out = self.fc(out) #Final Output
        return out

In [125]:
num_epochs = 10000 #1000 epochs
learning_rate = 0.001 #0.001 lr

input_size = 5 #number of features
hidden_size = 72 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers
seq_len = 24

num_classes = 2 #number of output classes 

In [126]:
lstm = LSTM(num_classes, input_size, hidden_size, num_layers, seq_len).to(DEVICE) #our lstm class 


In [127]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate) 

In [129]:
train_loss_arr = []
train_acc_arr = []
val_loss_arr = []
val_acc_arr = []
for epoch in range(num_epochs):
    total_correct = 0
    total_samples = 0
    train_loss = 0
    lstm.train()
    for X_batch, y_batch in train_loader:
        outputs = lstm.forward(X_batch) #forward pass
        predictions = torch.argmax(outputs, dim=1)

        optimizer.zero_grad() 
        loss = criterion(outputs, y_batch.long())

        loss.backward() #calculates the loss of the loss function

        optimizer.step() #improve from loss, i.e backprop
        total_correct += (predictions == y_batch).sum().item()
        total_samples += y_batch.size(0)
        train_loss += loss.item()
    train_loss_arr.append(train_loss)
     
    val_total_correct = 0
    val_total_samples = 0
    val_loss = 0
    lstm.eval()
    for X_val_batch, y_val_batch in val_loader:
        outputs = lstm.forward(X_val_batch) #forward pass
        predictions = torch.argmax(outputs, dim=1)
        loss = criterion(outputs, y_val_batch.long())


        val_total_correct += (predictions == y_val_batch).sum().item()
        val_total_samples += y_val_batch.size(0)
        val_loss += loss.item()
    val_loss_arr.append(val_loss)
    
    train_accuracy = 100 * total_correct / total_samples
    train_acc_arr.append(train_accuracy)   
    
    val_accuracy = 100 * val_total_correct / val_total_samples
    val_acc_arr.append(val_accuracy)   
    if epoch % 10 == 0:
        
        print("Epoch: %d, loss: %1.5f" % (epoch, train_loss) + ' : Acc = %1.3f' % train_accuracy + ' : T Acc = %1.3f' % val_accuracy)


Epoch: 0, loss: 79.58773 : Acc = 76.742 : T Acc = 78.103
Epoch: 10, loss: 79.65410 : Acc = 76.742 : T Acc = 78.103
Epoch: 20, loss: 79.62092 : Acc = 76.742 : T Acc = 78.103
Epoch: 30, loss: 79.58837 : Acc = 76.742 : T Acc = 78.103
Epoch: 40, loss: 79.56387 : Acc = 76.742 : T Acc = 78.103
Epoch: 50, loss: 79.52663 : Acc = 76.742 : T Acc = 78.103
Epoch: 60, loss: 79.52381 : Acc = 76.742 : T Acc = 78.103
Epoch: 70, loss: 79.42340 : Acc = 76.742 : T Acc = 78.103
Epoch: 80, loss: 79.36748 : Acc = 76.742 : T Acc = 78.103
Epoch: 90, loss: 79.35567 : Acc = 76.742 : T Acc = 78.103
Epoch: 100, loss: 79.28214 : Acc = 76.742 : T Acc = 78.103
Epoch: 110, loss: 79.29112 : Acc = 76.742 : T Acc = 78.103
Epoch: 120, loss: 79.07568 : Acc = 76.742 : T Acc = 78.103
Epoch: 130, loss: 78.97124 : Acc = 76.742 : T Acc = 78.103
Epoch: 140, loss: 78.87089 : Acc = 76.742 : T Acc = 78.103
Epoch: 150, loss: 78.91138 : Acc = 76.734 : T Acc = 78.103
Epoch: 160, loss: 78.62866 : Acc = 76.777 : T Acc = 78.088
Epoch: 1

KeyboardInterrupt: 

In [ ]:
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([0.1])).to('cuda')
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate, weight_decay=1e-5) 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lstm = lstm.to(device)

y_train_tensors = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)

X_train_tensors = X_train_tensors.to(device)
y_train_tensors = y_train_tensors.to(device)
batch_size = 256
batches_per_epoch = int(len(X_train_tensors) / batch_size)
for epoch in range(num_epochs):
    total_correct = 0
    total_samples = 0
    lstm.train()
    for i in range(batches_per_epoch):
        start = i * batch_size
        # take a batch
        Xbatch = X_train_tensors[start:start+batch_size]
        ybatch = y_train_tensors[start:start+batch_size]
        outputs = lstm.forward(Xbatch) #forward pass
#         predictions = torch.argmax(outputs, dim=1)
#         predictions = torch.round(outputs)
        predictions = (outputs > 0.0).float()
        optimizer.zero_grad() #caluclate the gradient, manually setting to 0
        # obtain the loss function
        loss = criterion(outputs, ybatch.float())

        loss.backward() #calculates the loss of the loss function

        optimizer.step() #improve from loss, i.e backprop
#         print(predictions)
#         print(ybatch)
#         print('====================')
        total_correct += (predictions == ybatch).sum().item()
#         print((predictions == ybatch))
        total_samples += ybatch.size(0)
        
    if epoch % 10 == 0:
        accuracy = 100 * total_correct / total_samples
        total_correct = 0
        total_samples = 0
        false_correct = 0
        positive_correct = 0

        lstm.eval()
        X_test_tensors = X_test_tensors.to(device)
        y_test_tensors = y_test_tensors.to(device)
        outputs = lstm.forward(X_test_tensors) #forward pass
        # predictions = torch.argmax(outputs, dim=1)
        predictions = (outputs > 0.0).float()
#         print(predictions)
#         print(y_test_tensors[y_test_tensors ==1])
        total_correct += (predictions == y_test_tensors).sum().item()
        false_correct += (predictions[y_test_tensors ==0] == y_test_tensors[y_test_tensors ==0]).sum().item()
        positive_correct += (predictions[y_test_tensors ==1] == y_test_tensors[y_test_tensors ==1]).sum().item()
        positive_samples = torch.count_nonzero(y_test_tensors)
        total_samples = y_test_tensors.size(0)
        negative_samples = total_samples -positive_samples

        test_accuracy = 100 * total_correct / total_samples
        p_test_accuracy = 100 * positive_correct / positive_samples
        n_test_accuracy = 100 * false_correct / negative_samples

        print("Ech: %d, ls: %1.5f" % (epoch, loss.item()) + ' : Acc = %2.2f' % accuracy + ' : T Acc = %2.2f' % test_accuracy + ' : TP Acc = %2.2f' % p_test_accuracy + ' : TN Acc = %2.4f' % n_test_accuracy)
